TASK-2

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df=pd.read_csv("Dataset .csv")

In [3]:
df=df.dropna() #handling missing values, deleting rows with missing values

In [4]:
df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                0
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [5]:
le=LabelEncoder()
df["City_encoded"]=le.fit_transform(df["City"])
df["Has Table booking_encoded"]=df["Has Table booking"].map({"Yes":1, "No":0})
df["Has Online delivery_encoded"]=df["Has Online delivery"].map({"Yes":1, "No":0})

df["Cuisines"] = df["Cuisines"].apply(lambda x: x.split(", ") if isinstance(x, str) else [])

mlb = MultiLabelBinarizer()# Apply MultiLabelBinarizer
cuisine_encoded = pd.DataFrame(mlb.fit_transform(df["Cuisines"]), columns=mlb.classes_)


df = pd.concat([df, cuisine_encoded], axis=1)# Concatenate df and cuisine_encoded

df.drop(columns=["Cuisines"], inplace=True)

In [6]:
all_cuisine_cols = mlb.classes_

In [7]:
df = df.dropna()

In [8]:
#create function for pre processesing the user input
def pre_process(df):
    le=LabelEncoder()
    df["City_encoded"]=le.fit_transform(df["City"])
    df["Has Table booking_encoded"]=df["Has Table booking"].map({"Yes":1, "No":0})
    df["Has Online delivery_encoded"]=df["Has Online delivery"].map({"Yes":1, "No":0})
    df["Cuisines"] = df["Cuisines"].apply(lambda x: x.split(", ") if isinstance(x, str) else [])

    mlb = MultiLabelBinarizer()
    cuisine_encoded = pd.DataFrame(mlb.fit_transform(df["Cuisines"]), columns=mlb.classes_)

    df = pd.concat([df, cuisine_encoded], axis=1)
    df.drop(columns=["Cuisines", "City", "Has Table booking", "Has Online delivery"], inplace=True)

    return df
    

In [9]:
features_df = pd.concat([
    cuisine_encoded,
    df[["City_encoded","Price range", "Has Online delivery_encoded", "Has Table booking_encoded"]]
], axis=1)

restaurant_names = df["Restaurant Name"]

In [10]:
features_df = features_df.dropna()

In [11]:
features_df.shape

(9533, 149)

In [12]:
user_input = {
    "Cuisines": ["Burger, Bar Food, Southern"],
    "City": ["Columbus"],
    "Price range": [2],
    "Has Online delivery": ["Yes"],
    "Has Table booking": ["No"]
}

In [13]:
input_df = pd.DataFrame.from_dict(user_input) #converting user input into dataframe

In [14]:
pp_input_df = pre_process(input_df)# preprocessing the user input

pp_input_df.shape

(1, 7)

In [15]:
#finding the missing values and assigning zero to it, because the two dfs are not aligned
missing_cols = [col for col in all_cuisine_cols if col not in pp_input_df.columns]

for missing_cuisine_col in missing_cols:
    pp_input_df[missing_cuisine_col] = 0

C:\Users\saiaa\AppData\Local\Temp\ipykernel_30880\1186183365.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pp_input_df[missing_cuisine_col] = 0
C:\Users\saiaa\AppData\Local\Temp\ipykernel_30880\1186183365.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pp_input_df[missing_cuisine_col] = 0
C:\Users\saiaa\AppData\Local\Temp\ipykernel_30880\1186183365.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [16]:
features_df.shape, pp_input_df.shape

((9533, 149), (1, 149))

In [17]:
#as the order of columns will be different in two dataframes so making it same as features_df
pp_input_df = pp_input_df[features_df.columns] 

In [18]:
similarities = cosine_similarity(pp_input_df, features_df)[0]#cosine similarity between the input and the features

top_indices = similarities.argsort()[::-1][:10]#selects top 10

recommended_restaurants = restaurant_names.iloc[top_indices]

recommendations = pd.DataFrame({
    "Restaurant": recommended_restaurants.values,
    "Similarity Score": similarities[top_indices]
})

print(recommendations)

                         Restaurant  Similarity Score
0            The Cheesecake Factory          0.750000
1   Sangeetha Vegetarian Restaurant          0.721688
2                   Bait El Khetyar          0.721688
3                       Hot Palayok          0.714435
4                            Gazebo          0.714435
5                         Via Delhi          0.714435
6  Cho Gao - Crowne Plaza Abu Dhabi          0.711512
7            Famous Dave's Barbecue          0.711512
8                        Applebee's          0.694365
9                  Indian By Nature          0.686406
